In [ ]:
from os import listdir
import os
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np


'''
Loading data
'''
start_dir = './tiny-imagenet-200/'

#List all catgories
imgCats = listdir(start_dir + 'train')
catNum = len(imgCats)
fileLocToNum = []
catToLabelDict = {}

#One-hot encoding
for i in range(catNum):
    catToLabelDict[imgCats[i]] = i
    for img in listdir(start_dir + 'train/' + imgCats[i] + '/images'):
        loc = start_dir + 'train/' + imgCats[i] + '/images/' + img
        fileLocToNum.append((loc, i))
        
from random import shuffle 
shuffle(fileLocToNum)
print(fileLocToNum[:10])


class training_batch:
    def __init__(self):
        pass
    
    def get(self):
        while 1:
            self.init_vars()
            for i in fileLocToNum:
                if(self.input_size < batch_size):
                    self.read_image(i)
                else:
                    yield self.x_train, self.y_train
                    self.init_vars()
                    self.read_image(i)
                self.input_size += 1
            yield self.x_train, self.y_train # for last amount of data
    
    def init_vars(self):
        self.input_size = 0
        self.x_train = np.empty([batch_size, 64, 64, 3]).astype(np.float32)
        self.y_train = np.empty([batch_size, catNum]).astype(np.float32)
    
    def read_image(self, i):
        entry = i
        img_arr = img_to_array(load_img(entry[0])) #channel first?
        if(img_arr.shape != (64, 64, 3)):
            img_arr = np.stack((img_arr, ) * 3)
        self.x_train[self.input_size] = img_arr / 255
        zeros = np.zeros(catNum)
        zeros[entry[1]] = 1.0
        self.y_train[self.input_size] = zeros

input_shape = (64, 64, 3)

class val_batch:
    def __init__(self):
        pass
            
    def get(self):
        f = open(start_dir + 'val/val_annotations.txt')
        while 1:
            self.init_vars()
            for pic_cat in f.readlines():
                #put into batch
                if(self.input_size < batch_size):
                    self.read_image(pic_cat.split())
                else:
                    yield self.x_val, self.y_val
                    self.init_vars()
                    self.read_image(pic_cat.split())
                self.input_size += 1
            yield self.x_val, self.y_val # for last amount of data
        f.close()
    
    def init_vars(self):
        self.input_size = 0
        self.x_val = np.empty([batch_size, 64, 64, 3]).astype(np.float32)
        self.y_val = np.empty([batch_size, catNum]).astype(np.float32)
    
    def read_image(self, i):
        entry = i
        file_name = entry[0]
        cat = entry[1]
        img_arr = img_to_array(load_img(start_dir + 'val/images/' + file_name))
        if(img_arr.shape != (64, 64, 3)):
            img_arr = np.stack((img_arr,) * 3) #change to 3-channel if black-white
        self.x_val[self.input_size] = img_arr / 255
        zeros = np.zeros(catNum)
        zeros[catToLabelDict[cat]] = 1.0
        self.y_val[self.input_size] = zeros

batch_size = 256
epochs = 100

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(catNum))
model.add(Activation('softmax'))

optimizer = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizer,
              metrics = ['categorical_accuracy'])

def get_training_batch():
    b = training_batch()
    return b.get()

def get_val_batch():
    b = val_batch()
    return b.get()

model.fit_generator(get_training_batch(),
                    steps_per_epoch = len(fileLocToNum) // batch_size + 1,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = get_val_batch(),
                    validation_steps = 10000 // batch_size + 1)

# Save model and weights
save_dir = './'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, 'test')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

score = model.evaluate(x_test, y_test, verbose=0) #沒test set 會爆
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import pickle
file_dir = './IMAGE_SET/train/train_data_batch_1'
with open(file_dir, 'rb') as fo:
    data = pickle.load(fo)

In [2]:
from os import listdir
from PIL import Image
import numpy as np
import pickle
import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
#from keras import backend as K
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size = 512
num_class = 1000
epochs = 50
input_shape = (64, 64, 3)

class batch:
    def __init__(self, set_name):
        self.dir = './IMAGE_SET/' + set_name + '/' #train or val
        pass
    
    def __read_file(self):
        for one_set in listdir(self.dir):
            with open(self.dir + one_set, 'rb') as fo:
                data = pickle.load(fo)
                x = data['data']
                y = data['labels']
                del(data)
                img_size = 64
                img_size2 = pow(64, 2)
                x = np.dstack((x[:, :img_size2], x[:, img_size2:2*img_size2], x[:, 2*img_size2:]))
                x = x.reshape((x.shape[0], img_size, img_size, 3))
                print(x.shape)
                y = [i-1 for i in y] #shift index to start from 0
                y = keras.utils.to_categorical(y, num_class)
                yield x, y
    
    def get_batch(self):
        for x_set, y_set in self.__read_file():
            counter = 0
            set_size = len(x_set)
            while (counter + 1) * batch_size < set_size:
                start = counter * batch_size
                end = (counter + 1) * batch_size
                yield x_set[start:end], y_set[start:end]
                counter += 1
            start = counter * batch_size
            yield x_set[start:set_size], y_set[start:set_size]    
    
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same',
                        input_shape=input_shape))
model.add(Activation('relu'))
#model.add(Conv2D(32, (3, 3)))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

#model.add(Conv2D(64, (3, 3), padding='same'))
#model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_class))
model.add(Activation('softmax'))

#optimizer = keras.optimizers.rmsprop(lr=0.001, decay=1e-5)
opt = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=true). 

model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

def get_training_batch():
    b = batch('train')
    return b.get_batch()

def get_val_batch():
    b = batch('val')
    return b.get_batch()

model.fit_generator(get_training_batch(),
                    steps_per_epoch = 1281167 // batch_size + 5, ##等等改喔
                    epochs = epochs,
                    verbose = 1,
                    validation_data = get_val_batch(),
                    validation_steps = 50000 // batch_size + 1)

# Save model and weights
save_dir = './'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, 'test')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Epoch 1/50
(128116, 64, 64, 3)
 169/2507 [=>............................] - ETA: 9:18 - loss: 16.0984 - acc: 0.0012

KeyboardInterrupt: 

In [ ]:
print(data['labels'][1])

In [ ]:
pow(64, 2)

In [ ]:
import numpy as np
a = np.arange(50)
print(a)
print(a[10:49])

In [4]:
from PIL import Image
import numpy as np
import pickle

with open('./IMAGE_SET/train/train_data_batch_1', 'rb') as fo:
    data = pickle.load(fo)
    x = data['data']
    y = data['labels']
    del(data)
    img_size = 64
    img_size2 = pow(64, 2)
    x = np.dstack((x[:, :img_size2], x[:, img_size2:2*img_size2], x[:, 2*img_size2:]))
    x = x.reshape((x.shape[0], img_size, img_size, 3))
    y = [i-1 for i in y] #shift index to start from 0
    
    index = 8787
    img = Image.fromarray(x[index])
    print(y[index])
    img.show()

680


In [10]:
import numpy as np
a = np.arange(100) * 0.1
arg = np.argwhere(a > 5)
print(a[arg])

[[ 5.1]
 [ 5.2]
 [ 5.3]
 [ 5.4]
 [ 5.5]
 [ 5.6]
 [ 5.7]
 [ 5.8]
 [ 5.9]
 [ 6. ]
 [ 6.1]
 [ 6.2]
 [ 6.3]
 [ 6.4]
 [ 6.5]
 [ 6.6]
 [ 6.7]
 [ 6.8]
 [ 6.9]
 [ 7. ]
 [ 7.1]
 [ 7.2]
 [ 7.3]
 [ 7.4]
 [ 7.5]
 [ 7.6]
 [ 7.7]
 [ 7.8]
 [ 7.9]
 [ 8. ]
 [ 8.1]
 [ 8.2]
 [ 8.3]
 [ 8.4]
 [ 8.5]
 [ 8.6]
 [ 8.7]
 [ 8.8]
 [ 8.9]
 [ 9. ]
 [ 9.1]
 [ 9.2]
 [ 9.3]
 [ 9.4]
 [ 9.5]
 [ 9.6]
 [ 9.7]
 [ 9.8]
 [ 9.9]]


In [14]:
y = [9,8,7,1,2,3]
y = np.asarray(y)
print(y)
y = [i-1 for i in y] #shift index to start from 0
train_args = np.argwhere(y < 5)
print(y[train_args])

[9 8 7 1 2 3]


TypeError: '<' not supported between instances of 'list' and 'int'